We will dissect each part of the backtester to see how they work.
# 1. The DataHandler

In [1]:
from data_handler import HistoricalPolygonDataHandler
from polygon.tickers import get_id
from datetime import datetime, date
from event import MarketEvent
import queue
import pandas as pd

In [2]:
events = queue.Queue()

In [3]:
data_handler = HistoricalPolygonDataHandler(events)
data_handler.load_data("AAPL", start=date(2023, 8, 1), end=date(2023, 9, 1))
data_handler.load_data("AA", start=date(2023, 7, 1), end=date(2023, 9, 1))

In [4]:
data_handler.get_loaded_symbols()

['AAPL', 'AA']

In [5]:
for i in range(10):  
    data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=30+i))

In [6]:
len(data_handler.latest_bars["AAPL"])

10

In [7]:
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:38:00,196.3950,196.49,196.3871,196.43,196.43,128663,True,False
2023-08-01 09:39:00,196.4205,196.59,196.3700,196.55,196.55,235643,True,False


# 2. Broker

In [8]:
from broker import SimulatedBroker
from event import OrderEvent
broker = SimulatedBroker(events, data_handler)
order = OrderEvent(datetime(2023, 8, 1, hour=9, minute=39), "AAPL", side="BUY", quantity=10)

2023-08-01T09:39:00 | ORDER BUY 10 of AAPL


In [9]:
broker.execute_order(order)

In [10]:
for i in range(10):
    events.get() # Remove the MarketEvents
event = events.get()
event

In [11]:
event.total_fill

1965.5

# 3. Portfolio
1. Initialize
2. Generate 3 MarketEvents
3. Buy 100 AAPL
4. Generate 3 MarketEvents
5. See the holdings
6. Sell 200 AAPL
7. See the holdings
8. Generate 3 MarketEvents
9. Buy 100 AAPL and finish

TDL: After this, finish portfolio. Then simplify the backtest loop.